In [43]:
import os
import pickle
import time

import streamlit as st

import langchain
from langchain_openai import OpenAI
from langchain_classic.chains import RetrievalQAWithSourcesChain
from langchain_classic.chains.qa_with_sources.loading import load_qa_with_sources_chain 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.globals import set_debug, set_verbose

from secret_key import openapi_key

In [44]:
os.environ['OPENAI_API_KEY'] = openapi_key

In [45]:
llm = OpenAI(temperature=0.9, max_tokens=500)

loaders = UnstructuredURLLoader(
    urls = [
        "https://business.bofa.com/en-us/content/market-strategies-insights/weekly-market-recap-report.html",
        "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
    ]
)

data = loaders.load()
len(data)

2

In [46]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

In [47]:
docs = text_splitter.split_documents(data)
len(docs)

18

In [48]:
embeddings = OpenAIEmbeddings()

In [49]:
vectorIndex_openai = FAISS.from_documents(docs,embeddings)

In [50]:
index_path = "faiss_vector_index"
vectorIndex_openai.save_local(index_path)
print(f"Vector store saved to {index_path}/")

Vector store saved to faiss_vector_index/


In [51]:
vectorIndex_openai = FAISS.load_local(
    "faiss_vector_index",
    embeddings,
    allow_dangerous_deserialization=True
)
print("Vector store loaded")

Vector store loaded


In [52]:
chain = RetrievalQAWithSourcesChain.from_llm(llm = llm, retriever = vectorIndex_openai.as_retriever(), verbose = True)
chain

RetrievalQAWithSourcesChain(verbose=True, combine_documents_chain=MapReduceDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question.\nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x3082e34d0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x306e7bf10>, temperature=0.9, max_tokens=500, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=True, combine_documents_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_types={}, partial_var

In [53]:
sample_query = "what is the price of Tiago iCNG?"
set_debug(True)
chain({"question": sample_query}, return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nfirst published: Aug 4, 2023 02:17 pm\n\nDiscover the latest Business News, Sensex, and Nifty updates. Obtain Personal Financ

{'answer': ' The price of the Tiago iCNG is between Rs 6.55 lakh and Rs 8.1 lakh.\n',
 'sources': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}